## LIME Test 2 Control Data Classification

In [1]:
# Import Libraries
import numpy as np
#import pandas as pd
from sklearn.model_selection import train_test_split

from control_data  import Control_Data
from feature_stats import Feature_Statistics, Class_Feature_Statistics


<class 'ImportError'>: cannot import name 'Class_Feature_Statistics' from 'feature_stats' (C:\Users\deneb\Ulster\Machine_Learning\MSc_Project\Lime\feature_stats.py)

In [ ]:
active_features  = ["Active_1", "Active_2", "Active_3", "Active_4", "Active_5"]
passive_features = ["Passive_1", "Passive_2", "Passive_3", "Passive_4", "Passive_5"]
all_features     = active_features + passive_features

In [ ]:
data = Control_Data(Y_std_dev=0.1,
                    Y_Func='Classification',
                    Active_X=active_features,
                    Passive_X=passive_features,
                    Num_Samples=500,
                    feature_bounds=[[0,2],[1,2],[2,3],[0,5],[8,10],\
                                    [0,2],[0,4],[0,5],[0,8],[0,10]])


In [ ]:
# Splitting X & y into training and testing set
X = data.Get_Features()
y = data.Get_Outcomes()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80)

print('Train Test Shapes:', X_train.shape, X_test.shape)


In [ ]:
#Train and Test a Random Forest with the data
from sklearn.ensemble import RandomForestClassifier
RF_Model = RandomForestClassifier(n_estimators=100, criterion='log_loss')
RF_Model.fit(X_train, y_train)

In [ ]:
#Train and Test a Linear Classifier with the data
from sklearn.linear_model import LogisticRegression

LR_Model = LogisticRegression()
LR_Model.fit(X_train, y_train)

In [ ]:
#Train and Test a Dummy Classifier with the data
#from sklearn.dummy import DummyClassifier 
#Dumb_Model = DummyClassifier (strategy = 'uniform')
#Dumb_Model.fit(X_train, y_train)

In [ ]:
print('Model Score Random Forest', RF_Model.score(X_test, y_test))
#print('Model Score Linear Reg.  ', LR_Model.score(X_test, y_test))
#print('Model Score Dummy        ', Dumb_Model.score(X_test, y_test))

In [ ]:
print('RF Feature Importances:  ', RF_Model.feature_importances_)
#print('LR Feature Coefficients: ', LR_Model.coef_)
#print('LR Feature No.Features:  ', LR_Model.n_features_in_)
#print('Dumb Output Classes:     ', Dumb_Model.classes_)
#print('Dumb Class Prior:        ', Dumb_Model.class_prior_)

In [ ]:
# LIME Explanation for Training Data
from lime import lime_tabular

RF_LIME = lime_tabular.LimeTabularExplainer(training_data=X_train,
                                            mode="classification",
                                            #training_labels=None,
                                            feature_names=data.Get_Feature_Names(),
                                            #categorical_features=None,
                                            #categorical_names=None,
                                            #kernel_width=None,
                                            #kernel=None,
                                            verbose=False,
                                            class_names=data.Get_Class_Names(),
                                            #feature_selection='auto',
                                            discretize_continuous=False,
                                            #discretizer='quartile',
                                            sample_around_instance=True)
                                            #random_state=None,
                                            #training_data_stats=None)


In [ ]:
# Generate explantion data for Random Foreall_featuresst

RF_Stats = Feature_Statistics(all_features, classes=['Zero', 'One'])

for test_instance in range(X_test.shape[0]):

    data_row  = X_test[test_instance,:]
    
    RF_explain_inst = RF_LIME.explain_instance(data_row=data_row,
                                               predict_fn=RF_Model.predict_proba,
                                               #labels=None,
                                               #top_labels=None,
                                               #num_features=10,
                                               num_samples=100,
                                               #distance_metric='euclidean',
                                               #model_regressor=None,
                                               sampling_method='gaussian')
    
    data_row  = data_row
    
    y_predict = RF_Model.predict_proba(data_row.reshape(1, -1))

    print('INSTANCE: ', test_instance)
    
    RF_Stats.Add_Sample(sample     = RF_explain_inst.as_list(),
                        outcome    = y_test[test_instance],
                        prediction = y_predict)
    
    last_instance = test_instance

In [ ]:
print(RF_explain_inst.as_list())

print(y_predict)

In [ ]:
RF_explain_inst.show_in_notebook()

In [ ]:
RF_Stats.Feature_Counts(max_features=8, scaled=True, threshold=0.075)

In [ ]:
RF_Stats.Frequency_Plot()

In [ ]:
RF_Stats.Frequency_Plot(top_features=False)

In [ ]:
RF_Stats.Box_Plot()

In [ ]:
RF_Stats.Box_Plot(top_features=False)

In [ ]:
RF_Stats.Violin_Plot()

In [ ]:
RF_Stats.Violin_Plot(top_features=False)

In [ ]:
RF_Stats.View_Explanation(instance=last_instance)

In [ ]:
RF_Stats.View_Explanation(instance=last_instance, max_features=6)

In [ ]:
Stats = Classes_Container(RF_Stats)